In [ ]:
#!pip install pandas
#!pip install import_ipynb
#!pip install random

import pandas as pd
import import_ipynb

import os
exna = ""
for i in os.listdir():
    if i.startswith("Question Number CO Mapping"):
        exna = i

In [ ]:
def get_CO_mapping(subject_name):
    df_CO = pd.read_excel(exna,sheet_name="Sheet1")
    df_CO.head()
    index_of_subject = 0
    for i in range(len(df_CO)):
        if str(df_CO.iloc[i,0]) == subject_name:
            index_of_subject = i
    whole_List = []
    for ctr in range(1,4):
        CO_List = []
        for i in df_CO.iloc[index_of_subject+ctr,:]:
            if pd.isna(i):
                break
            CO_List.append(i)
        whole_List.append(CO_List)
    
    print(whole_List)
    return whole_List

In [ ]:
def map_CO(excel_file_name):
    df_Paper = pd.read_excel(excel_file_name)
    subject_name = ""
    flag = 0
    for i in excel_file_name[-5::-1]:
        if i==" " and flag == 0:
            subject_name+=""
            flag = 1
        elif i==" " and flag == 1:
            subject_name = subject_name[::-1].replace(" ","/")
            break
        subject_name+=i
    
    CO_List = get_CO_mapping(subject_name)
    
    total_no_students = len(df_Paper.loc[1:,1])
    
    from xlrd import open_workbook
    from xlutils.copy import copy

    rb = open_workbook(excel_file_name)

    wb = copy(rb)

    s = wb.get_sheet(0)

    i = 0
    if 'FIAT' in excel_file_name: i = 0
    elif 'SIAT' in excel_file_name: i = 1
    elif 'MODEL' in excel_file_name: i = 2

    for myRow in range(total_no_students+2,total_no_students+3):
        j = 0
        for myCol in range(1,len(CO_List[i])+1):
            s.write(myRow,myCol,CO_List[i][j])
            j+=1

    wb.save(excel_file_name[:-4]+".xls")

In [ ]:
def adjust_16m(excel_file_name):
    import xlrd
    from xlrd import open_workbook
    from xlutils.copy import copy
    from random import randint

    rb = open_workbook(excel_file_name)
    
    sh = rb.sheet_by_index(0)
    
    wb = copy(rb)

    s = wb.get_sheet(0)

    length = 0
    mark = []
    total_mark = []
    for i in range(1,sh.nrows):
        if sh.cell_value(i, 22)=='':
            length = i 
            break
        mark.append(sh.cell_value(i, 22))
        total_mark.append(sh.cell_value(i, 23))

    i = 0
    col = 22
    a16,b16,a1660,b1660 = 0,0,0,0 
    for row in range(1,length):
        aORb = randint(0,1)
        if mark[i] == 'AB':
            s.write(row,col,'AB')
            s.write(row,col+1,'AB')
        elif aORb == 0:
            s.write(row,col,mark[i])
            s.write(row,col+1,0)
            if mark[i]>0:
                a16+=1
            if mark[i]>9:
                a1660+=1
        else:
            s.write(row,col,0)
            s.write(row,col+1,mark[i])
            if mark[i]>0:
                b16+=1
            if mark[i]>9:
                b1660+=1
                
        s.write(row,col+2,total_mark[i])
        i+=1
    
    s.write(length+1,22,"16a")
    s.write(length+2,22,a16)
    s.write(length+3,22,a1660)
    s.write(length+4,22,a1660/a16*100)
    s.write(length+1,23,"16b")
    s.write(length+2,23,b16)
    s.write(length+3,23,b1660)
    s.write(length+4,23,b1660/b16*100)
    s.write(0,22,'16a')
    s.write(0,23,"16b")
    s.write(0,24,"Total Marks")

    wb.save(excel_file_name)

In [ ]:
import os
myList = []
for i in os.listdir():
    if i.startswith("With"):
        myList.append(i)
for i in myList:
    if "With Part C 100" in i:
        adjust_16m(i)
    map_CO(i)
    print("\nMapping CO for "+str(i)+" ...\n")